In [1]:
import tensorflow as tf


gpus = tf.config.list_physical_devices('GPU')

print(f'Detected gpus: {gpus}')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print('Set dynamic GPU memory allocation.')

Detected gpus: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Set dynamic GPU memory allocation.


2022-04-30 23:19:41.605920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 23:19:41.610598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 23:19:41.611109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [ ]:
import wandb
os.environ["WANDB_NOTEBOOK_NAME"] = "colorgan.ipynb"
wandb.init(project="colorgan", tags=["gan"])


In [2]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from utils import PROJ_ROOT, LogPredictionsCallback
from models import get_unet_generator, get_discriminator, ColorGan
from dataset import postprocess, folder_dataset

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from wandb.keras import WandbCallback

In [3]:
g = get_unet_generator()

2022-04-30 23:19:42.469274: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-30 23:19:42.469911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 23:19:42.470524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 23:19:42.471002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, 

In [4]:
d = get_discriminator() 

In [5]:
gan = ColorGan(g, d)

gan.compile(
    d_optimizer=Adam(2e-4, beta_1=0.5),
    g_optimizer=Adam(2e-4, beta_1=0.5),
    end_loss=BinaryCrossentropy(label_smoothing=0.2),
)

In [6]:
BATCH_SIZE = 128
PREFETCH = tf.data.AUTOTUNE

ds_train = folder_dataset(
    f"{PROJ_ROOT}/imagenet/ILSVRC/Data/CLS-LOC/train",
    augment=True,
    img_size=(512, 512),
    batch_size=BATCH_SIZE,
    prefetch=PREFETCH,
)

ds_monitor = (
    folder_dataset(
        f"{PROJ_ROOT}/imagenet/ILSVRC/Data/CLS-LOC/val",
        augment=False,
        img_size=(512, 512),
        batch_size=1,
    )
    .unbatch()
    .take(2000)
    .shuffle(buffer_size=2000, seed=42)
    .take(128)
    .batch(BATCH_SIZE)
)


Found 864 files belonging to 1 classes.
Found 11 files belonging to 1 classes.


2022-04-30 23:19:43.635247: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [14]:
model_name = "gan_v00_epoch{epoch:02d}"
model_path = f"{PROJ_ROOT}/models/{model_name}"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_freq="epoch",
)

wandb_callback = WandbCallback()

visualization_callback = LogPredictionsCallback(ds_monitor, every_n_batch=1000)

In [18]:
history = gan.fit(
    ds_train,
    epochs=10,
    verbose=1,
    callbacks=[
        model_checkpoint_callback,
        wandb_callback,
        visualization_callback,
    ]
)

Epoch 1/100
3/3 [==============================] - 1s 234ms/step - d_loss: 1.3160 - g_loss: 23.6896 - val_loss: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 236ms/step - d_loss: 1.2962 - g_loss: 23.4452 - val_loss: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 228ms/step - d_loss: 1.2774 - g_loss: 23.2307 - val_loss: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 1s 224ms/step - d_loss: 1.2467 - g_loss: 22.9863 - val_loss: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 1s 235ms/step - d_loss: 1.2311 - g_loss: 22.8380 - val_loss: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 1s 243ms/step - d_loss: 1.2072 - g_loss: 22.7531 - val_loss: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 1s 241ms/step - d_loss: 1.1718 - g_loss: 22.4077 - val_loss: 0.0000e+00
Epoch 8/100
3/3 [==============================] - 1s 226ms/step - d_loss: 1.2074 - g_loss: 22.1861 - val_loss: 0.0000e+00
Epoch 9/100
3/3 